In [1]:
import sys
print("Python Version:", sys.version)

Python Version: 3.7.4 (default, Aug 13 2019, 20:35:49) 
[GCC 7.3.0]


In [2]:
#######################
# standard code block #
#######################

%config InlineBackend.figure_format = 'svg'

# auto reload imports that change
%load_ext autoreload
# only set to auto reload for marked imports
%autoreload 1

In [37]:
import pandas as pd

### Connect to Database

In [8]:
from pymongo import MongoClient
%aimport credentials.cred
from credentials import cred


config = {
  'host': cred.mongo_host,
  'username': cred.mongo_user,
  'password': cred.mongo_pass,
  'authSource': cred.mongo_auth_db
}

# get a mongo client
client = MongoClient(**config)

# use the raw database
jre_raw = client.jre_raw
podcasts_raw = jre_raw.podcasts

# use the clean database
jre_clean = client.jre_clean
podcasts_clean = jre_clean.podcasts

### Clean One Podscribe

In [11]:
one_ep = podcasts_raw.find_one({'source':'podscribe'})['text']

In [39]:
ep_mongo = list(podcasts_raw.find({'source':'podscribe'},{'_id':0, 'text':1, 'name':1}).limit(10))
ep_list = [x['text'] for x in ep_mongo]
ep_names = [x['name'] for x in ep_mongo]

In [ ]:
# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [ ]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [35]:
ep_list

[" Hello friends this episode the podcast is brought to you by the motherfucking cash app the cash app the simplest way for you to send and save money and now the simplest way for you to try to grow that money introducing cash app investing cash app investing unlike bullshit-ass Investments tools that only let you buy entire shares of stock the cash app lets you instantly invest as little or as much as you want  this way when your favorite company stock is just a little too expensive you can still own a piece with as little as one dollar whoo and because the cash app is directly connected to your bank account there is no four or five day waiting periods for inbound transfers so you can start investing today brokerage services are provided by cash app investing a subsidiary of square and a member of s IPC and of course when you download the cash app enter the referral code Joe  Rogan all one word you will receive ten dollars and the cash app will send ten dollars to Justin Ren's fight f

### Vectorize

In [40]:
# Vectorize
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(ep_list)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = ep_names
data_dtm

,00,000,08,10,100,101,11,1111,11th,12,...,zippers,zombie,zone,zoned,zones,zoo,zoom,zooms,zoos,zurich
#1383 - Malcolm Gladwell,1,2,1,12,1,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1
#1383 - Malcolm Gladwell,1,2,1,12,1,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1
#1382 - RZA & Donnell Rawlings,0,1,0,15,6,0,1,1,1,4,...,0,0,1,2,0,0,0,1,0,0
#1378 - Greg Fitzsimmons,2,7,0,7,1,0,0,0,0,1,...,3,0,4,0,1,0,0,0,0,0
#1375 - Edward Norton,0,0,0,1,2,1,1,0,0,2,...,0,0,1,0,0,0,0,1,0,0
#1373 - Kyle Kulinski,0,13,0,5,7,2,1,0,1,0,...,0,0,0,0,0,0,1,0,0,0
#1371 - Andrew Santino,2,6,0,4,1,0,2,0,0,4,...,0,0,0,0,0,0,0,0,0,0
#1370 - Brian Grazer,7,1,0,11,2,0,1,0,0,3,...,0,0,2,0,0,0,0,0,0,0
#1369 - Christopher Ryan,1,2,0,3,1,0,0,0,0,2,...,0,0,0,0,0,2,0,0,1,0
#1368 - Edward Snowden,0,0,0,6,5,0,9,0,4,0,...,0,0,0,0,0,0,0,0,0,0
